<a href="https://colab.research.google.com/github/Lomzem/LomzemStockQuant/blob/main/LomzemStocks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import os
import pandas as pd
import datetime
import sys

wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'

def get_stock_data(date, ticker):
	api_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/minute/{date}/{date}?adjusted=true&sort=asc&limit=50000&apiKey=IyEWxobQmnBJh0QwQeNOWH7QggrJGFjp'
	try:
		data = requests.get(api_url).json()
		stock_df = pd.DataFrame(data['results'])
		stock_df.rename(columns = {'c': 'Close', 'h': 'High', 'l': 'Low', 'o': 'Open', 'v': 'Volume', 't': 'Time'}, inplace=True)
		stock_df.insert(0, 'Ticker', ticker)
		stock_df.insert(1, 'FTime', stock_df.apply(lambda row: pd.to_datetime(row['Time']-1.44e+7, unit='ms'), axis=1))
		stock_df.to_excel(f'{raw_stock_data_path}/{date}-{ticker}.xlsx', index=False)
		print(f'Got {raw_stock_data_path}/{date}-{ticker}.xlsx')
	except KeyError:
		print('Wait a little longer')
		print(date, ticker)
		sys.exit()

def read_wanted_sheet():
	df = pd.read_excel(wanted_sheet_path)
	for index, row in df.iterrows():
		files = [file for file in os.listdir(raw_stock_data_path)]
		if not (f'{row["Date"].date()}-{row["Ticker"]}.xlsx') in files:
			if row['Date'].date() < datetime.date.today()-datetime.timedelta(days=1):
				get_stock_data(row['Date'].date(), row['Ticker'].upper())

read_wanted_sheet()

Got /content/drive/MyDrive/LomzemStockCharts/rawstockdata/2022-03-15-WAFU.xlsx
Got /content/drive/MyDrive/LomzemStockCharts/rawstockdata/2022-03-16-WAFU.xlsx
Got /content/drive/MyDrive/LomzemStockCharts/rawstockdata/2022-03-17-WAFU.xlsx


In [4]:
# import pandas as pd
import os

cum_stock_data = pd.DataFrame()

for file in os.listdir('/content/drive/MyDrive/LomzemStockCharts/rawstockdata'):
	df = pd.read_excel(f'/content/drive/MyDrive/LomzemStockCharts/rawstockdata/{file}')
	cum_stock_data = pd.concat([cum_stock_data, df])

print(cum_stock_data)
cum_stock_data.to_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx', index=False)

    Ticker               FTime   Volume      vw  Open  Close  High   Low  \
0     VERU 2022-04-11 06:30:00   7448.0  5.1801  4.91   5.47  5.48  4.55   
1     VERU 2022-04-11 06:31:00   6869.0  5.3946  5.20   5.45  5.93  5.16   
2     VERU 2022-04-11 06:32:00  49308.0  6.0368  5.42   5.72  6.53  5.42   
3     VERU 2022-04-11 06:33:00  38057.0  5.6815  5.73   5.71  5.90  5.50   
4     VERU 2022-04-11 06:34:00  29283.0  5.6577  5.75   5.70  5.83  5.51   
..     ...                 ...      ...     ...   ...    ...   ...   ...   
216   WAFU 2022-03-17 15:59:00  11948.0  3.1691  3.17   3.17  3.17  3.15   
217   WAFU 2022-03-17 16:02:00    300.0  3.2500  3.25   3.25  3.25  3.25   
218   WAFU 2022-03-17 16:24:00    301.0  3.1800  3.18   3.18  3.18  3.18   
219   WAFU 2022-03-17 17:09:00    142.0  3.1800  3.18   3.18  3.18  3.18   
220   WAFU 2022-03-17 17:59:00   1000.0  3.1900  3.19   3.19  3.19  3.19   

              Time    n   a  op  
0    1649673000000   55 NaN NaN  
1    1649673060000 

In [6]:
import requests
import os
import pandas as pd
import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df = pd.read_excel('/content/drive/MyDrive/LomzemStockCharts/cumulative_stock_data.xlsx')
ddf = pd.DataFrame()

ddf = pd.concat([ddf, df[['Ticker', 'FTime']]])
ddf['FTime'] = ddf['FTime'].dt.date

# df.loc[192]
# Get Open
t_df = df.loc[df['FTime'].dt.time==datetime.time(9,30), ['Ticker', 'FTime', 'Open']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Close
t_df = df.loc[df['FTime'].dt.time==datetime.time(15,59), ['Ticker', 'FTime', 'Close']]
t_df['FTime'] = t_df['FTime'].dt.date
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['High'].transform(max) == t_df['High']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Low
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(9,30)) & (df['FTime'].dt.time<=datetime.time(15,59))]
t_df = t_df[['Ticker', 'FTime', 'Low']]
t_df['FTime'] = t_df['FTime'].dt.date
idx = t_df.groupby(['Ticker', 'FTime'])['Low'].transform(min) == t_df['Low']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<=datetime.time(20,0))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df = t_df.groupby(['Ticker', 'FTime'])['Volume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Premarket Volume
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,0)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'Volume']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'Volume': 'PVolume'}, inplace=True)
t_df = t_df.groupby(['Ticker', 'FTime'])['PVolume'].sum()
t_df.drop_duplicates(inplace=True)
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])

# Get Gap %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['Gap%'] = (ddf['Open'] / ddf.groupby('Ticker').shift(1)['Close'] - 1) * 100

# Get Premarket High
t_df = df.loc[(df['FTime'].dt.time>=datetime.time(4,00)) & (df['FTime'].dt.time<datetime.time(9,30))]
t_df = t_df[['Ticker', 'FTime', 'High']]
t_df['FTime'] = t_df['FTime'].dt.date
t_df.rename(columns={'High': 'PHigh'}, inplace=True)
idx = t_df.groupby(['Ticker', 'FTime'])['PHigh'].transform(max) == t_df['PHigh']
t_df = t_df[idx].drop_duplicates()
ddf = ddf.merge(t_df, on=['Ticker', 'FTime'])
ddf.insert(5, 'PHigh', ddf.pop('PHigh'))

# Get Change From Open %

ddf['ChangeFromOpen%'] = (ddf['Close']/ddf['Open']-1)*100

# Get HOD - PrevClose %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['HOD-PrevClose%'] = (ddf['High'] / ddf.groupby('Ticker').shift(1)['Close'] - 1) * 100

# Get HOD - PrevHOD %
ddf.sort_values(['Ticker', 'FTime'], inplace=True)
ddf.drop_duplicates(inplace=True)
ddf['HOD-PrevHOD%'] = (ddf['High'] / ddf.groupby('Ticker').shift(1)['High'] - 1) * 100

# Get Volume Ratio
ddf['Vol Ratio'] = ddf['Volume']/ddf['PVolume']

ddf.rename(columns={'FTime': 'Date'}, inplace=True)

ddf['closedRed'] = ddf['Close'] < ddf['Open']

### PATTERN: Overextended Gap Down

def overextended_gap_down():
  ddf['isOEGD'] = False
  prev_change_from_open = ddf.groupby('Ticker').shift(1)['ChangeFromOpen%']
  ddf.loc[(prev_change_from_open > 30) & (ddf['Gap%']<0), ['isOEGD']] = True

### PATTERN: Gap Up Short

def gap_up_short():
  ddf['isGUS'] = False
  ddf.loc[(ddf['Gap%'] > 35) & (ddf['closedRed']==True) & (ddf['Low'] / ddf['Open']-1 <= -0.1), ['isGUS']] = True

overextended_gap_down()
gap_up_short()

ddf
# print(ddf.head())
# print(ddf.groupby('Ticker')['Close'].shift(1).head())
# t_df

,Ticker,Date,Open,Close,High,PHigh,Low,Volume,PVolume,Gap%,ChangeFromOpen%,HOD-PrevClose%,HOD-PrevHOD%,Vol Ratio,closedRed,isOEGD,isGUS
0,ADGI,2022-03-29,3.84,3.85,4.03,3.95,3.82,1403665.00,7888.00,NaN,0.26,NaN,NaN,177.95,False,False,False
1,ADGI,2022-03-30,5.73,5.04,6.14,7.11,5.00,92056322.00,18393238.00,48.83,-12.04,59.48,52.36,5.00,True,False,True
2,ADGI,2022-03-31,4.99,4.56,4.99,5.70,4.53,8306119.00,1052830.00,-1.09,-8.53,-0.99,-18.73,7.89,True,False,False
3,ADN,2022-04-05,2.22,2.01,2.28,2.26,1.99,1249714.00,2330.00,NaN,-9.46,NaN,NaN,536.36,True,False,False
4,ADN,2022-04-06,2.58,3.63,3.98,2.66,2.47,162103715.00,1049987.00,28.36,40.89,98.01,74.56,154.39,False,False,False
5,ADN,2022-04-07,3.32,3.30,3.45,3.88,3.04,19160827.00,2447205.00,-8.67,-0.60,-5.09,-13.32,7.83,True,True,False
6,ADN,2022-04-08,3.36,2.94,3.55,3.39,2.80,9323953.00,404272.00,1.82,-12.50,7.58,2.90,23.06,True,False,False
7,ADN,2022-04-11,2.81,2.68,2.83,2.92,2.57,2888275.00,131866.00,-4.43,-4.63,-3.74,-20.28,21.90,True,False,False
8,AXSM,2022-04-19,40.26,40.92,41.61,46.80,38.05,11045166.00,2037628.00,NaN,1.64,NaN,NaN,5.42,False,False,False
9,BBAI,2022-04-05,8.79,10.39,11.27,8.32,8.79,2093598.00,125.00,NaN,18.26,NaN,NaN,16748.78,False,False,False


In [3]:
import os
import pandas as pd

def search_for_duplicates_in_wanted():
    wanted_sheet_path = '/content/drive/MyDrive/LomzemStockCharts/stocks_to_obtain.xlsx'
    raw_stock_data_path = '/content/drive/MyDrive/LomzemStockCharts/rawstockdata'
    df = pd.read_excel(wanted_sheet_path)
    df.drop_duplicates(inplace = True)
    df.dropna(inplace=True)
    df.sort_values(['Ticker'])
    df['Date'] = df['Date'].dt.date
    df.to_excel(wanted_sheet_path, index = False)

search_for_duplicates_in_wanted()